In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import re

# Read data from Excel file
data = pd.read_excel("Operations_Dataset_Dummy.xlsx")

# Clean up column names
data.columns = [re.sub(r'\W+', '', col) for col in data.columns]

# Separate 'Company' column and feature columns
companies = data['Company']
features = data.drop(columns=['Company'])

# Step 1: Standardize the feature data
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# Step 2: Fit the PCA model
pca = PCA(n_components=features.shape[1])  # Set the number of components equal to the number of features
pca.fit(features_standardized)

# Step 3: Extract the principal components and their corresponding weights
principal_components = pca.components_
explained_variance_ratio = pca.explained_variance_ratio_

# Calculate average weights for each feature
average_weights = np.abs(principal_components).mean(axis=0) * np.sqrt(explained_variance_ratio)

# Normalize weights so that the sum equals 1
normalized_weights = average_weights / np.sum(average_weights)

# Define weights for each variable based on feature importance scores
weights = {}
for i, column in enumerate(features.columns):
    weights[column] = normalized_weights[i]

# Define conditional mappings for compliance scoring based on variable values
conditional_mappings = {
    'FragilityStateIndex': lambda value: 1 - value / features['FragilityStateIndex'].max(),  # Inverse relationship with fragility state index
    'AvgNaturalDisasterFrequencyperyear': lambda value: 1 - value / features['AvgNaturalDisasterFrequencyperyear'].max(),  # Inverse relationship with natural disaster frequency
    'GeographicDiversity': lambda value: 1 if value == 1 else 0,
    'DataOwnershipandReturn': lambda value: 1 if value == 1 else 0,
    'BusinessContinuityPlans': lambda value: 1 if value == 1 else 0,
    'TransitionAssistance': lambda value: 1 if value == 1 else 0,
    'SingleSourcing': lambda value: 1 if value == 1 else 0,
    'DisasterRecoveryPlans': lambda value: 1 if value == 1 else 0,
    'RedundancyandFailoverMechanisms': lambda value: 1 if value == 1 else 0,
    'IncidentResponseandCrisisManagement': lambda value: 1 if value == 1 else 0
}

# Calculate operations score for each company
operations_scores = {}
for i, row in data.iterrows():
    company = row['Company']
    score = 0
    for column in features.columns:
        # Apply conditional mapping to adjust operations score based on variable value
        score += weights[column] * conditional_mappings[column](row[column])
    # Ensure the score is a single value
    operations_scores[company] = score

# Reverse and scale the scores
scaled_operations_scores = {company: (1 - score) * 100 for company, score in operations_scores.items()}

# Create a DataFrame from the scaled_compliance_scores dictionary
scaled_operations_scores_df = pd.DataFrame(list(scaled_operations_scores.items()), columns=['Company', 'Scaled operations Score'])

# Save the DataFrame to a CSV file
scaled_operations_scores_df.to_csv("scaled_operations_scores_24mar.csv", index=False)

In [2]:
weights

{'FragilityStateIndex': 0.13548697891936906,
 'AvgNaturalDisasterFrequencyperyear': 0.12220487142103865,
 'GeographicDiversity': 0.12552415088145924,
 'DataOwnershipandReturn': 0.11373965854460093,
 'BusinessContinuityPlans': 0.11164390149084703,
 'TransitionAssistance': 0.10204833853541168,
 'SingleSourcing': 0.08949102152195829,
 'DisasterRecoveryPlans': 0.07308592018692822,
 'RedundancyandFailoverMechanisms': 0.07314689665973557,
 'IncidentResponseandCrisisManagement': 0.053628261838651305}